In [9]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model
from RBM import rbm_train, rbm_reconstruct, rbm_reconstruction
from main import gen_adaptive

In [2]:
#pieces_major = multi_training.loadPieces('music/major')
#pieces_minor = multi_training.loadPieces('music/minor')
path = 'C_music'
pieces = multi_training.loadPieces(dirpath = path)

Loaded preprocessed notes...


In [3]:
m = model.Model([300, 300], [100, 50], dropout=0.5)

model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [4]:
m.learned_config = pickle.load(open("output/params11000.p", "rb"))

In [5]:
rbm = rbm_train(np.array(dict(pieces.items()[:]).values()), window_len = 16, learning_rate=0.1,
                training_epochs=1, batch_size=20, n_hidden=1500)

         +25 files trained on, time (m:s) - 5:45
         +25 files trained on, time (m:s) - 11:11
         +25 files trained on, time (m:s) - 15:19
         +25 files trained on, time (m:s) - 19:12
         +25 files trained on, time (m:s) - 22:16
         +25 files trained on, time (m:s) - 25:27
         +25 files trained on, time (m:s) - 28:40
         +25 files trained on, time (m:s) - 33:3
         +25 files trained on, time (m:s) - 37:56
         +25 files trained on, time (m:s) - 42:30
         +25 files trained on, time (m:s) - 49:47
Training epoch 0, cost is -128.335205078, time (m:s) - 52:44
Training took 47.121287 minutes


In [11]:
pickle.dump(rbm, open("output/RBM.pkl", "wb"))

In [38]:
gen_adaptive(m,dict(pieces.items()),5,name="composition_11000_4:46", rbm=rbm)

(99996,)
RBM reconstruction: 0/97500. Time (m:s) - 15:23
(99996,)
RBM reconstruction: 156/97500. Time (m:s) - 15:25
(99996,)
RBM reconstruction: 312/97500. Time (m:s) - 15:26
(99996,)
RBM reconstruction: 468/97500. Time (m:s) - 15:27
(99996,)
RBM reconstruction: 624/97500. Time (m:s) - 15:28
(99996,)
RBM reconstruction: 780/97500. Time (m:s) - 15:29
(99996,)
RBM reconstruction: 936/97500. Time (m:s) - 15:30
(99996,)
RBM reconstruction: 1092/97500. Time (m:s) - 15:31
(99996,)
RBM reconstruction: 1248/97500. Time (m:s) - 15:32
(99996,)
RBM reconstruction: 1404/97500. Time (m:s) - 15:34
(99996,)
RBM reconstruction: 1560/97500. Time (m:s) - 15:35
(99996,)
RBM reconstruction: 1716/97500. Time (m:s) - 15:36
(99996,)
RBM reconstruction: 1872/97500. Time (m:s) - 15:37
(99996,)
RBM reconstruction: 2028/97500. Time (m:s) - 15:38
(99996,)
RBM reconstruction: 2184/97500. Time (m:s) - 15:39
(99996,)
RBM reconstruction: 2340/97500. Time (m:s) - 15:40
(99996,)
RBM reconstruction: 2496/97500. Time (m:

In [52]:
#%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [5]:
batches = 7000
batches_old = 7000

print 'Training {0}+{1} batches on {2}'.format(batches,batches_old, path)

multi_training.trainPiece(m, pieces, [batches, batches_old])#, notes_to_input = None)
# add starting output names from given batch
pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(1*batches), "wb"))

Training 7000+7000 batches on C_music
epoch 7000, error=3910.11181641
epoch 7100, error=3201.51464844
epoch 7200, error=3557.14331055
epoch 7300, error=3301.77880859
epoch 7400, error=3411.85083008
epoch 7500, error=2850.02050781
epoch 7600, error=2729.79711914
epoch 7700, error=3236.95825195
epoch 7800, error=3338.45605469
epoch 7900, error=2618.94360352
epoch 8000, error=2448.27905273
epoch 8100, error=2741.92114258
epoch 8200, error=3192.16162109
epoch 8300, error=3195.8269043
epoch 8400, error=2547.27685547
epoch 8500, error=2688.00341797
epoch 8600, error=2603.19702148
epoch 8700, error=2600.62866211
epoch 8800, error=1943.21704102
epoch 8900, error=2440.41845703
epoch 9000, error=2308.94799805
epoch 9100, error=2478.546875
epoch 9200, error=3259.46582031
epoch 9300, error=2789.42773438
epoch 9400, error=2606.84716797
epoch 9500, error=2881.77661133
epoch 9600, error=2081.78613281
epoch 9700, error=2299.42285156
epoch 9800, error=3328.75219727
epoch 9900, error=2487.84521484
epoch

In [47]:
from __future__ import print_function

from datetime import datetime

import numpy as np
import six.moves.cPickle as pickle
import theano
import theano.tensor as T
import os
from theano.tensor.shared_randomstreams import RandomStreams
from RBM import share_data, RBM

train_data = np.array(pieces.values())
window_len = 16
learning_rate=0.1
batch_size=20
n_hidden=1500

# allocate symbolic variables for the data
index = T.lscalar()
x = T.matrix('x')  # the data is presented as rasterized images
beat_stride = 78*2
rng = np.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))

# initialize storage for the persistent chain (state = hidden layer of chain)
persistent_chain = theano.shared(np.zeros((batch_size, n_hidden), dtype=theano.config.floatX), borrow=True)

# construct the RBM class
rbm = RBM(input=x, n_visible=window_len*beat_stride,
          n_hidden=n_hidden, numpy_rng=rng, theano_rng=theano_rng)

# get the cost and the gradient corresponding to one step of CD-15
cost, updates = rbm.get_cost_updates(lr=learning_rate,
                                     persistent=persistent_chain,
                                     k=15)

#################################
#     Training the RBM          #
#################################

# start-snippet-5
# it is ok for a theano function to have no output
# the purpose of train_rbm is solely to update the RBM parameters


def share_train(data):
    shared_ = share_data(make_windows(data))
    return shared_

def make_windows(data):
    data = data.reshape((-1, 78 * 2))
    data = np.array([data[idx:idx + window_len] for idx in range(len(data) - window_len)])
    data = data.reshape((-1, 16 * 78 * 2))
    return data.astype('float32')

train_set = share_train(train_data[0])

train_rbm = theano.function(
    [index],
    cost,
    updates=updates,
    givens={
        x: train_set[index:index+batch_size]
    },
    name='train_rbm'
)

def train(training_epochs):
    # go through training epochs
    for epoch in range(training_epochs):
        # go through the training set
        mean_cost = []
        for t in range(0,len(train_data)):
            #tm = datetime.now()
            train_set.set_value(make_windows(train_data[t]))
            #print('Time wasted on data compilation :', datetime.now()-tm)
            idxs = train_data[t].shape[0]
            for idx in range(0,idxs-batch_size,batch_size):
                mean_cost += [train_rbm(idx)]
                #idx = random.choice(range(len(train_set)))
                #piece_output = train_set.values()[idx]
                #start = random.randrange(0,len(piece_output)-window_len,division_len)
            if t % 25 == 0:
                tm = datetime.now()
                print('         +25 files trained on, time (m:s) - {0}:{1}'.format(tm.minute, tm.second))
        tm = datetime.now()
        print('Training epoch {0}, cost is {1}, time (m:s) - {2}:{3}'.format(epoch, np.mean(mean_cost), tm.minute, tm.second))

In [48]:
training_epochs = 1
train(training_epochs)

         +25 files trained on, time (m:s) - 44:54
         +25 files trained on, time (m:s) - 49:45
         +25 files trained on, time (m:s) - 53:59
         +25 files trained on, time (m:s) - 57:39
         +25 files trained on, time (m:s) - 1:39
         +25 files trained on, time (m:s) - 6:6
         +25 files trained on, time (m:s) - 9:59
         +25 files trained on, time (m:s) - 14:29
         +25 files trained on, time (m:s) - 18:5
         +25 files trained on, time (m:s) - 22:2
         +25 files trained on, time (m:s) - 32:54
Training epoch 0, cost is -116.842918396, time (m:s) - 39:13


In [50]:
train(training_epochs)

         +25 files trained on, time (m:s) - 43:59
         +25 files trained on, time (m:s) - 48:44
         +25 files trained on, time (m:s) - 52:44
         +25 files trained on, time (m:s) - 56:12
         +25 files trained on, time (m:s) - 59:47
         +25 files trained on, time (m:s) - 3:30
         +25 files trained on, time (m:s) - 6:59
         +25 files trained on, time (m:s) - 11:28
         +25 files trained on, time (m:s) - 15:3
         +25 files trained on, time (m:s) - 18:42
         +25 files trained on, time (m:s) - 26:19
Training epoch 0, cost is -86.5165634155, time (m:s) - 30:25


In [ ]:
train(15)
#reduce gibbs step, so that it doesn't converge too much into single pattern

         +25 files trained on, time (m:s) - 0:10
         +25 files trained on, time (m:s) - 5:36
         +25 files trained on, time (m:s) - 10:30
         +25 files trained on, time (m:s) - 14:23
         +25 files trained on, time (m:s) - 18:23
         +25 files trained on, time (m:s) - 22:40
         +25 files trained on, time (m:s) - 26:40

In [ ]:
def reconstruct_multiple(dirpath='reconstruction', pieces_fname ='/pieces.pkl'):
    pieces = {}
    pieces_fname = dirpath+pieces_fname
    ignores = 0
    for fname in os.listdir(dirpath):
        if fname.lower()[-4:] != '.mid' or 'rbm' in fname.lower():
            continue
        outMatrix = midiToNoteStateMatrix(os.path.join(dirpath, fname))
        rbm_outputs, rbm_outputs_discrete = rbm_reconstruct(rbm, np.array(outMatrix), window_len = 16)

        noteStateMatrixToMidi(np.array(rbm_outputs), 'reconstruction/' + fname + ' rbm')
        noteStateMatrixToMidi(np.array(rbm_outputs_discrete), 'reconstruction/' + fname + ' discrete rbm')
        print('Saved generated music and reconstruction.')

reconstruct_multiple()

(20124,)
RBM reconstruction: 0/17628. Time (m:s) - 35:54
(20124,)
RBM reconstruction: 156/17628. Time (m:s) - 35:55
(20124,)
RBM reconstruction: 312/17628. Time (m:s) - 35:56
(20124,)
RBM reconstruction: 468/17628. Time (m:s) - 35:57
(20124,)
RBM reconstruction: 624/17628. Time (m:s) - 35:58
(20124,)
RBM reconstruction: 780/17628. Time (m:s) - 35:59
(20124,)
RBM reconstruction: 936/17628. Time (m:s) - 36:1
(20124,)
RBM reconstruction: 1092/17628. Time (m:s) - 36:2
(20124,)
RBM reconstruction: 1248/17628. Time (m:s) - 36:3
(20124,)
RBM reconstruction: 1404/17628. Time (m:s) - 36:4
(20124,)
RBM reconstruction: 1560/17628. Time (m:s) - 36:5
(20124,)
RBM reconstruction: 1716/17628. Time (m:s) - 36:6
(20124,)
RBM reconstruction: 1872/17628. Time (m:s) - 36:8
(20124,)
RBM reconstruction: 2028/17628. Time (m:s) - 36:9
(20124,)
RBM reconstruction: 2184/17628. Time (m:s) - 36:10
(20124,)
RBM reconstruction: 2340/17628. Time (m:s) - 36:11
(20124,)
RBM reconstruction: 2496/17628. Time (m:s) - 36: